# Chapter 7 The Command Pattern



**Command Pattern**也是屬於**Behavioral Pattern**主要是針對於物件相關的操作，把方法明確定義出來，使用命令的方式來操作。其中實作上會有的組件大致上有

 - Command

觸發Receiver物件的方法

 - Receiver

接受命令，並執行動作

 - Invoker
 
知道如何執行相對應的command

 - Client
 
負責建立Command物件

**Command Pattern**主要用途是，將request包裝成物件，如此對於request可朔性就會變高！ 比如request排入queue中。書中使用一件安裝程式來當例子。
 

In [3]:
class Wizard:
    
    def __init__(self, src, root_dir):
        self.choices = []
        self.root_dir = root_dir
        self.src = src
        
    def preferences(self, command):
        self.choices.append(command)
        
    def execute(self):
        for choice in self.choices:
            if list(choice.values())[0]:
                print(f"Copying binaries -- {self.src} to {self.root_dir}")
            else:
                print("No operation")
                
wizard = Wizard("python3.5.gzip", "/usr/bin")
wizard.preferences({"Python": True})
wizard.preferences({"Java": False})
wizard.execute()

Copying binaries -- python3.5.gzip to /usr/bin
No operation


上面是個簡單的示範，將command排入queue，並依序執行相對應的動作。 繼續讀後發現，顯然關於**Command pattern**上面幾個物件應該是作者自己分的，我後來研究並沒有規定這個多東西，但是他的核心主軸有表達到。

In [4]:
import abc

class Command(metaclass=abc.ABCMeta):
    
    def __init__(self, recv):
        self.recv = recv
        
    def execute(self):
        pass
    

class ConcreteCommand(Command):
    
    def __init__(self, recv):
        self.recv = recv
        
    def execute(self):
        self.recv.action()
        
class Receiver:
    
    def action(self):
        print("Receiver Action")
        
class Invoker:
    
    def command(self, cmd):
        self.cmd = cmd
        
    def execute(self):
        self.cmd.execute()
        
recv = Receiver()
cmd = ConcreteCommand(recv)
invoker = Invoker()
invoker.command(cmd)
invoker.execute()

Receiver Action


上面是一個抽象化表達，跟第一個例子比，她多了用Receiver將命令的執行做了一層包裝，好處是這樣以後要替換多了彈性。接著書中舉個現實世界的例子來講解**Command Pattern**，股票交易中心

In [15]:
class Order(metaclass=abc.ABCMeta):
    
    @abc.abstractmethod
    def execute(self):
        pass
    
class BuyStockOrder(Order):
    
    """commander"""
    
    def __init__(self, stock):
        self.stock = stock
        
    def execute(self):
        self.stock.buy()
        
class SellStockOrder(Order):
    
    """commander"""
    
    def __init__(self, stock):
        self.stock = stock
        
    def execute(self):
        self.stock.sell()
        

class StockTrade:
    
    """Receiver """
    
    def buy(self):
        print("you will buy stocks")
        
    def sell(self):
        print("you will sell stocks")
        
        
class Agent:
    
    """Invoker"""
    
    def __init__(self):
        self._orderQueue = []
        
    def placeOrder(self, order):
        self._orderQueue.append(order)
        order.execute()
        
    
stock = StockTrade()
buyStock = BuyStockOrder(stock)
sellStock = SellStockOrder(stock)
agent = Agent()
agent.placeOrder(buyStock)
agent.placeOrder(sellStock)

you will buy stocks
you will sell stocks


這裡的invoker(class Agent)其實可以稍微修改，讓大家知道他將command放入queue中，並拿出來做事這樣，要不然書中這樣感覺那個queue根本多餘...

In [17]:
import threading
import time

class Agent:
    
    def __init__(self):
        self._orderQueue = []
        self._lock = threading.Condition()
        
        t = threading.Thread(target=self.schedule_run)
        t.start()
        
    def placeOrder(self, order):
        with self._lock:
            self._orderQueue.append(order)
            self._lock.notify()
        
    def schedule_run(self):
        
        while True:
            self._lock.acquire()
        
            if len(self._orderQueue) == 0:
                self._lock.wait()
            
            order = self._orderQueue.pop() 
            order.execute()
        
            self._lock.release()
            time.sleep(0.5)
            print(f'content is {self._orderQueue}')
        

if __name__ == "__main__":
    stock = StockTrade()
    buyStock = BuyStockOrder(stock)
    sellStock = SellStockOrder(stock)
    agent = Agent()
    agent.placeOrder(buyStock)
    agent.placeOrder(sellStock)

you will sell stocks
content is [<__main__.BuyStockOrder object at 0x10ce0f940>]
you will buy stocks
content is []


上面是個稍微示範一下放入queue中處理的程式碼，另外關於前面不是有講到其實很多人的**Command Pattern**定義並沒有那麼多嗎？ 在書中章節的最後確實也有人提問，作者也說確實可以不用這麼多定義